In [1]:
import pandas as pd
import numpy as np
import os.path
import math
from keras import Sequential
from keras.layers import Dense, LSTM

Using TensorFlow backend.


In [2]:
# read in data
fiction = pd.read_csv("fiction.csv", delimiter=",")
isbnToInfo = pd.read_csv("isbnToInfo.csv", delimiter=",")
genreData = pd.read_csv("sortedGenresFiction.csv", delimiter=",")
genreData.fillna(0, inplace=True)  # replace na with 0

In [3]:
def getMatrix(given_csv, k):
    sorted = {}

    # format into dictionary
    for i in range(given_csv.shape[0]):  # per day
        for j in range(1, 21):  # per entry
            if given_csv.iloc[i, j] in sorted:  # if key exist
                sorted[given_csv.iloc[i, j]].append([given_csv.iloc[i, 0], j])
            else:  # key doesn't exist
                sorted[given_csv.iloc[i, j]] = [[given_csv.iloc[i, 0], j]]

    # remove ones that have less than k entries since hard to predict
    clean = {}
    weeks = {}
    for i in sorted.keys():
        if len(sorted[i]) >= k:
            clean[i] = []
            weeks[i] = []
            for j in sorted[i]:
                clean[i].append(j[1])
                weeks[i].append(j[0])

    isbns = list(clean.keys())

    print(clean)

    # add first 3
    nX = []
    nY = []
    nList = []
    maeList = []
    goodISBNs = []
    badEntriesIndex = []

    for i in clean.keys():
        nX.append(clean[i][0:3])

    print('nx length = ', len(nX))

    # graphing a specific book
    wannaCheckIsbn = '0525952926'
    wannaCheckIsbnIndex = 0
    wannaCheckIsbnPredictions = []
    wannaCheckIsbnActual = clean[wannaCheckIsbn][0:k]

    # model
    for n in range(3, k):

        params = []
        nList.append(n)
        nY.clear()

        for j in clean.keys():
            nY.append(clean[j][n])
            goodISBNs.append(j)

        for i in range(len(nX)):  # per book

            # ranking
            # slope, intercept, r_value, p_value, std_err = stats.linregress(list(range(len(nX[i]))), nX[i])
            model = np.polyfit(list(range(len(nX[i]))), nX[i], 2)
            last = nX[i][len(nX[i]) - 1]
            slope = model[0]
            slope1 = model[1]

            # genre
            curISBN = isbns[i]

            listI = isbnToInfo[['isbn']].values.tolist()
            flattenedI = [item for sublist in listI for item in sublist]
            curi = flattenedI.index(curISBN)
            curGenre = isbnToInfo[['Category']].iloc[curi][0]

            if "/" in curGenre:  # take first genre if there are 2
                curGenre = curGenre.split("/")[0]
            prevGenrePercent = []
            listG = fiction[['date']].values.tolist()
            flattened = [item for sublist in listG for item in sublist]
            curWeek = flattened.index(weeks[curISBN][0])

            for j in range(curWeek - 3, curWeek):
                prevGenrePercent.append(genreData[[curGenre]].iloc[j][0])

            # gSlope, gIntercept, gR_value, gP_value, gStd_err = stats.linregress(list(range(len(prevGenrePercent))), prevGenrePercent)
            model = np.polyfit(list(range(len(prevGenrePercent))), prevGenrePercent, 2)
            gSlope0 = model[0]
            gSlope1 = model[1]

            # searches
            curISBNnoZero = curISBN[1:]
            if os.path.isfile(
                    "datadump/" + curISBN + ".csv"):  # for now, until we get all the data
                curSearchesDirty = pd.read_csv(
                    "datadump/" + curISBN + ".csv",
                    delimiter=",")  # cuz not clean
            elif os.path.isfile("datadump/" + curISBNnoZero + ".csv"):
                curSearchesDirty = pd.read_csv(
                    "datadump/" + curISBNnoZero + ".csv", delimiter=",")
            else:
                goodISBNs.remove(curISBN)
                # if n > 3:
                #     print('suprise!')
                badEntriesIndex.append(i)
                continue
            curSearchesBusty = curSearchesDirty[['GT_SearchIndex']].values.tolist()  # busty cuz not flat
            curSearches = [item for sublist in curSearchesBusty for item in sublist]  # flatten
            if len(curSearches) == 0:  # if file empty
                badEntriesIndex.append(i)
                continue

            count = 0
            for j in curSearches:
                if j == 0:  # if searches that day is 0
                    count += 1

            if count / len(curSearches) > 0.25:  # if more than 25% of the searches is 0, change to weekly
                curSearchesWeekly = []
                count = 0
                accumalativeSearches = 0
                totalIterations = 0
                for j in curSearches:
                    # for j in curSearches[4:]:

                    totalIterations += 1
                    count += 1
                    accumalativeSearches += j
                    if count == 7:  # every 7 days (1 week), we record the sales that week
                        if totalIterations < 7 * (k + 4):  # cuz start data starts from 1 month before
                            curSearchesWeekly.append(accumalativeSearches)
                            accumalativeSearches = 0
                            count = 0

            elif count / len(curSearches) <= 0.25:
                curSearchesWeekly = []
                totalIterations = 0

                for j in curSearches:
                    # for j in curSearches[4:]:
                    if totalIterations < 7 * (k + 4):
                        totalIterations += 1
                        curSearchesWeekly.append(j)  # well its rly daily searches but too lazy to make another var

            if len(curSearchesWeekly) < k:
                badEntriesIndex.append(i)


    print(len(nX))

    #####################################
    #####################################
    #####################################
    #####################################
    #####################################
    #####################################

    matrix = []
    bullshitassbullshit = []
    sorted = {}

    # format into dictionary
    for i in range(given_csv.shape[0]):  # per day
        for j in range(1, 21):  # per entry
            if given_csv.iloc[i, j] in sorted:  # if key exist
                sorted[given_csv.iloc[i, j]].append([given_csv.iloc[i, 0], j])
            else:  # key doesn't exist
                sorted[given_csv.iloc[i, j]] = [[given_csv.iloc[i, 0], j]]

    # remove ones that have less than k entries since hard to predict
    clean = {}
    weeks = {}
    for i in sorted.keys():
        if len(sorted[i]) >= k:
            clean[i] = []
            weeks[i] = []
            for j in sorted[i]:
                clean[i].append(j[1])
                weeks[i].append(j[0])

    isbns = list(clean.keys())

    # add first 3
    nX = []
    nY = []
    nList = []
    maeList = []
    # goodISBNs = []
    badEntriesIndex = []

    for i in clean.keys():
        nX.append(clean[i][0:k])


    xlength = len(clean.keys())
    ylength = k
    zlength = 3

    for a in range(xlength):  # num of books

        if a in badEntriesIndex:
            continue

        tempArray2 = []
        for b in range(ylength):  # num of weeks
            tempArray = []

            for c in range(3):
                tempArray.append(0)
            tempArray2.append(tempArray)
        matrix.append(tempArray2)

    matriy = []
    for a in range(xlength):  # num of books
        # matriy.append([None]*3)
        matriy.append([None]*1)

        # matriy.append([])
    print(np.shape(matrix))
    print(np.shape(matriy))

    variable = 0
    skipNextTurn = False
    numOfBadCount = 0
    for a in range(xlength):  # num of books
    # while variable < range(xlength):

        if a in badEntriesIndex:
            # variable = variable-1
            numOfBadCount += 1
            continue

        #####################################
        # genre
        curISBN = isbns[a-numOfBadCount]

        listI = isbnToInfo[['isbn']].values.tolist()
        flattenedI = [item for sublist in listI for item in sublist]
        curi = flattenedI.index(curISBN)
        curGenre = isbnToInfo[['Category']].iloc[curi][0]

        if "/" in curGenre:  # take first genre if there are 2
            curGenre = curGenre.split("/")[0]
        listG = fiction[['date']].values.tolist()
        flattened = [item for sublist in listG for item in sublist]
        curWeek = flattened.index(weeks[curISBN][0])

        # searches
        curISBNnoZero = curISBN[1:]
        if os.path.isfile(
                "datadump/" + curISBN + ".csv"):  # for now, until we get all the data
            curSearchesDirty = pd.read_csv(
                "datadump/" + curISBN + ".csv",
                delimiter=",")  # cuz not clean
        elif os.path.isfile("datadump/" + curISBNnoZero + ".csv"):
            curSearchesDirty = pd.read_csv(
                "datadump/" + curISBNnoZero + ".csv", delimiter=",")
        # else:
        #     # goodISBNs.remove(curISBN)
        #     badEntriesIndex.append(a)
        #     continue
        curSearchesBusty = curSearchesDirty[['GT_SearchIndex']].values.tolist()  # busty cuz not flat
        curSearches = [item for sublist in curSearchesBusty for item in sublist]  # flatten
        # if len(curSearches) == 0:  # if file empty
        #     badEntriesIndex.append(a)
        #     continue

        count = 0
        for j in curSearches:
            if j == 0:  # if searches that day is 0
                count += 1

        # if count / len(curSearches) > 0.25 and len(curSearches)/7 >= k:  # if more than 25% of the searches is 0, change to weekly
        # if True:
        #     curSearchesWeekly = []
        #     count = 0
        #     accumalativeSearches = 0
        #     totalIterations = 0
        #     howMany = math.floor(len(curSearches) / k)
        #     for j in curSearches[4:(7*k + 4)]:
        #         # for j in curSearches[4:]:
        #
        #         totalIterations += 1
        #         count += 1
        #         accumalativeSearches += j
        #         if count == howMany:  # every 7 days (1 week), we record the sales that week
        #             # if totalIterations < 7 * (k + 4):  # cuz start data starts from 1 month before
        #             if True:
        #                 curSearchesWeekly.append(accumalativeSearches)
        #                 accumalativeSearches = 0
        #                 count = 0
        curSearchesWeekly = []
        for j in range(k):
            howMany = math.floor(len(curSearches)/k)
            cumalative = 0
            for l in curSearches[(j*howMany): (j*(howMany+1))]:
                cumalative += l
            curSearchesWeekly.append(cumalative)

        # elif count / len(curSearches) <= 0.25:
        #     curSearchesWeekly = []
        #     totalIterations = 0
        #
        #     for j in curSearches[4:]:
        #         # for j in curSearches[4:]:
        #         if totalIterations < 7 * (k + 4):
        #             totalIterations += 1
        #             curSearchesWeekly.append(j)  # well its rly daily searches but too lazy to make another var


        #####################################

        for b in range(ylength):  # num of weeks
            # past, genre, google

            # print(matrix)
            if b < k-1:
                matrix[a-numOfBadCount][b][0] = nX[a-numOfBadCount][b]
                matrix[a-numOfBadCount][b][1] = genreData[[curGenre]].iloc[curWeek][0]
                try:
                    # print(b, curSearchesWeekly)
                    matrix[a-numOfBadCount][b][2] = curSearchesWeekly[b]
                except:
                    # del matrix[a][b]
                    # print(b, len(curSearchesWeekly))
                    bullshitassbullshit.append([a, b])
            else:
                # matriy[a][0] = nX[a][b]
                # matriy[a][1] = genreData[[curGenre]].iloc[curWeek][0]
                # matriy[a][2] = curSearchesWeekly[b]
                # print(nX[a])
                # print(nX[a][b])
                # matriy[a][0] = nX[a]
                numOfBadCount2 = 0
                for l in range(len(nX)):
                    if l in badEntriesIndex:
                        numOfBadCount2 += 1
                    if l not in badEntriesIndex:
                        try:
                            matriy[l-numOfBadCount2] = nX[l][k-1]
                        except:
                            print(nX[l])
                            print(nX[l][k])
                            print(matriy[l - numOfBadCount2])

        # variable += 1
    #print(bullshitassbullshit)
    # for i in reversed(bullshitassbullshit):
    #     print(i[0])
    #     print(i[1])

        # del matrix[i[0]][i[1]]

    # print(matrix)
    # print(np.shape(matrix))
    
    matrix = np.asarray(matrix)
    matriy = np.asarray(matriy)
    print(np.shape(matrix))
    print(np.shape(matriy))
    # print(matrix)
    #print(matriy)
    return matrix, matriy

In [4]:
matrix, matriy = getMatrix(fiction, 10)

{'0316068047': [2, 2, 2, 3, 3, 5, 6, 7, 4, 3, 3, 5, 6, 5, 5, 6, 6, 7, 10, 11, 9, 11, 12, 12, 13, 15, 17, 10, 8, 5, 5, 5, 3, 2, 2, 2, 2, 4, 3, 4, 8, 6, 4, 4, 5, 2, 4, 5, 6, 8, 8, 7, 7, 9, 11, 13, 17, 15, 16, 17, 18, 16, 12, 16, 19, 19, 17, 17], '0312348673': [3, 4, 4, 5, 5, 10, 11, 10, 13, 15, 14, 13, 15, 18], '0061537934': [13, 14, 15, 13, 15, 17, 17, 17, 17, 19, 20], '0743290119': [6, 6, 6, 4, 8, 13, 12, 10, 10, 10, 11, 18], '0061374229': [14, 9, 6, 5, 4, 2, 2, 4, 6, 7, 8, 6, 9, 9, 1, 1, 2, 2, 3, 5, 6, 9, 11, 10, 7, 4, 4, 4, 4, 6, 5, 5, 3, 3, 6, 7, 8, 13, 16, 20], '0385340990': [5, 3, 4, 3, 3, 5, 3, 5, 9, 12, 15, 16, 15, 13, 16, 17, 13, 18, 18, 16, 15, 19, 17, 10, 8, 8, 9, 8, 8, 6, 10, 10, 11, 7, 9, 13, 13, 9, 14, 15], '0307269752': [4, 7, 10, 12, 13, 17, 17, 20, 19, 17, 18, 20], '0446579939': [1, 1, 2, 3, 4, 6, 9, 11, 9, 12, 8, 7, 7, 10, 16, 16, 17], '0743270428': [1, 2, 5, 7, 12, 16, 16, 17, 18, 17], '0446533424': [1, 4, 6, 8, 12, 15, 14, 13, 13, 16, 19], '0446195502': [1, 3, 5, 8, 

150
(150, 10, 3)
(150, 1)
(150, 10, 3)
(150,)


In [5]:
inputX = []
for j in range(3, 9): #to predict j+1th result
    if j!=3:
        for i in range(np.shape(matrix)[0]):
            inputX[i].append([inputY[i], matrix[i][j-2][1], matrix[i][j-2][2]])

IndexError: list index out of range

In [ ]:
inputX = []
inputY = []

#initialize
for i in range(np.shape(matrix)[0]):
    tempX = []
    for j in range(3):
        tempX2 = []
        for k in range(3):
            tempX2.append(matrix[i][j][k])
        tempX.append(tempX2)
    inputX.append(tempX)
    inputY.append(matrix[i][3][0])

#print(inputY)

for j in range(3, 9): #to predict j+1th result
    if j!=3:
        for i in range(np.shape(matrix)[0]):
            inputX[i].append([inputY[i], matrix[i][j-1][1], matrix[i][j-1][2]])
            
    inputX = np.asarray(inputX)          
    model = Sequential()
    model.add(LSTM(units=30, return_sequences=True, input_shape=(inputX.shape[1], 3)))
    model.add(LSTM(units=30, return_sequences=True))
    model.add(LSTM(units=30))
    model.add(Dense(units=1))
    print(model.summary())
    
    print('MODEL IS TRAINING TO PREDICT THE ' + str(j+1) + 'TH RANK')
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    model.fit(inputX, inputY, epochs=400, batch_size=32)
    
    inputY = model.predict(inputX)
    inputY = inputY.tolist()
    inputY = [item for sublist in inputY for item in sublist]
    
    inputX = inputX.tolist()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 3, 30)             4080      
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 30)             7320      
_________________________________________________________________
lstm_3 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 18,751
Trainable params: 18,751
Non-trainable params: 0
_________________________________________________________________
None
MODEL IS TRAINING TO PREDICT THE 4TH RANK
Instructions for updating:
Use tf.cast instead.
Epoch 1/400
150/150 [==============================] - 4s 24ms/step - loss: 50.1256 - acc: 0.0000e+00
E

150/150 [==============================] - 0s 304us/step - loss: 2.4217 - acc: 0.3600
Epoch 69/400
150/150 [==============================] - 0s 327us/step - loss: 2.3238 - acc: 0.3400
Epoch 70/400
150/150 [==============================] - 0s 274us/step - loss: 2.3100 - acc: 0.3533
Epoch 71/400
150/150 [==============================] - 0s 267us/step - loss: 2.2167 - acc: 0.3400
Epoch 72/400
150/150 [==============================] - 0s 292us/step - loss: 2.1813 - acc: 0.3467
Epoch 73/400
150/150 [==============================] - 0s 330us/step - loss: 2.2187 - acc: 0.3333
Epoch 74/400
150/150 [==============================] - 0s 327us/step - loss: 2.2228 - acc: 0.3333
Epoch 75/400
150/150 [==============================] - 0s 314us/step - loss: 2.1955 - acc: 0.3333
Epoch 76/400
150/150 [==============================] - 0s 324us/step - loss: 2.1112 - acc: 0.3267
Epoch 77/400
150/150 [==============================] - 0s 357us/step - loss: 2.0137 - acc: 0.3667
Epoch 78/400
150/150 [=

150/150 [==============================] - 0s 370us/step - loss: 1.0451 - acc: 0.3533
Epoch 151/400
150/150 [==============================] - 0s 334us/step - loss: 1.0258 - acc: 0.4067
Epoch 152/400
150/150 [==============================] - 0s 284us/step - loss: 1.0034 - acc: 0.3667
Epoch 153/400
150/150 [==============================] - 0s 331us/step - loss: 1.0655 - acc: 0.3800
Epoch 154/400
150/150 [==============================] - 0s 361us/step - loss: 1.0785 - acc: 0.3467
Epoch 155/400
150/150 [==============================] - 0s 334us/step - loss: 1.0465 - acc: 0.4000
Epoch 156/400
150/150 [==============================] - 0s 334us/step - loss: 0.9948 - acc: 0.3667
Epoch 157/400
150/150 [==============================] - 0s 315us/step - loss: 1.0214 - acc: 0.4200
Epoch 158/400
150/150 [==============================] - 0s 321us/step - loss: 0.9963 - acc: 0.3533
Epoch 159/400
150/150 [==============================] - 0s 341us/step - loss: 0.9819 - acc: 0.3667
Epoch 160/400


150/150 [==============================] - 0s 271us/step - loss: 0.7134 - acc: 0.4667
Epoch 233/400
150/150 [==============================] - 0s 390us/step - loss: 0.6923 - acc: 0.4267
Epoch 234/400
150/150 [==============================] - 0s 314us/step - loss: 0.7105 - acc: 0.4133
Epoch 235/400
150/150 [==============================] - 0s 283us/step - loss: 0.7107 - acc: 0.4467
Epoch 236/400
150/150 [==============================] - 0s 324us/step - loss: 0.7568 - acc: 0.3667
Epoch 237/400
150/150 [==============================] - 0s 331us/step - loss: 0.7658 - acc: 0.4267
Epoch 238/400
150/150 [==============================] - 0s 324us/step - loss: 0.6814 - acc: 0.4533
Epoch 239/400
150/150 [==============================] - 0s 298us/step - loss: 0.6701 - acc: 0.4533
Epoch 240/400
150/150 [==============================] - 0s 271us/step - loss: 0.6661 - acc: 0.4400
Epoch 241/400
150/150 [==============================] - 0s 334us/step - loss: 0.6812 - acc: 0.4400
Epoch 242/400


150/150 [==============================] - 0s 321us/step - loss: 0.4736 - acc: 0.5467
Epoch 315/400
150/150 [==============================] - 0s 354us/step - loss: 0.4674 - acc: 0.5667
Epoch 316/400
150/150 [==============================] - 0s 344us/step - loss: 0.4722 - acc: 0.5467
Epoch 317/400
150/150 [==============================] - 0s 301us/step - loss: 0.4749 - acc: 0.5333
Epoch 318/400
150/150 [==============================] - 0s 372us/step - loss: 0.4775 - acc: 0.5733
Epoch 319/400
150/150 [==============================] - 0s 308us/step - loss: 0.4802 - acc: 0.5667
Epoch 320/400
150/150 [==============================] - 0s 337us/step - loss: 0.4550 - acc: 0.5267
Epoch 321/400
150/150 [==============================] - 0s 337us/step - loss: 0.4559 - acc: 0.5600
Epoch 322/400
150/150 [==============================] - 0s 354us/step - loss: 0.4684 - acc: 0.5467
Epoch 323/400
150/150 [==============================] - 0s 330us/step - loss: 0.4842 - acc: 0.5667
Epoch 324/400


150/150 [==============================] - 0s 284us/step - loss: 0.4182 - acc: 0.6267
Epoch 397/400
150/150 [==============================] - 0s 288us/step - loss: 0.4146 - acc: 0.5400
Epoch 398/400
150/150 [==============================] - 0s 321us/step - loss: 0.4589 - acc: 0.5467
Epoch 399/400
150/150 [==============================] - 0s 324us/step - loss: 0.5917 - acc: 0.4733
Epoch 400/400
150/150 [==============================] - 0s 281us/step - loss: 0.4839 - acc: 0.5333
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 4, 30)             4080      
_________________________________________________________________
lstm_5 (LSTM)                (None, 4, 30)             7320      
_________________________________________________________________
lstm_6 (LSTM)                (None, 30)                7320      
____________________________________________________

150/150 [==============================] - 0s 410us/step - loss: 1.4951 - acc: 0.0000e+00
Epoch 67/400
150/150 [==============================] - 0s 410us/step - loss: 1.4837 - acc: 0.0000e+00
Epoch 68/400
150/150 [==============================] - 0s 421us/step - loss: 1.4361 - acc: 0.0000e+00
Epoch 69/400
150/150 [==============================] - 0s 400us/step - loss: 1.3830 - acc: 0.0000e+00
Epoch 70/400
150/150 [==============================] - 0s 433us/step - loss: 1.3177 - acc: 0.0000e+00
Epoch 71/400
150/150 [==============================] - 0s 386us/step - loss: 1.2804 - acc: 0.0000e+00
Epoch 72/400
150/150 [==============================] - 0s 397us/step - loss: 1.2379 - acc: 0.0000e+00
Epoch 73/400
150/150 [==============================] - 0s 370us/step - loss: 1.1827 - acc: 0.0000e+00
Epoch 74/400
150/150 [==============================] - 0s 404us/step - loss: 1.1586 - acc: 0.0000e+00
Epoch 75/400
150/150 [==============================] - 0s 423us/step - loss: 1.1145 -

150/150 [==============================] - 0s 403us/step - loss: 0.2167 - acc: 0.0000e+00
Epoch 145/400
150/150 [==============================] - 0s 393us/step - loss: 0.2101 - acc: 0.0000e+00
Epoch 146/400
150/150 [==============================] - 0s 423us/step - loss: 0.2065 - acc: 0.0000e+00
Epoch 147/400
150/150 [==============================] - 0s 393us/step - loss: 0.2019 - acc: 0.0000e+00
Epoch 148/400
150/150 [==============================] - 0s 393us/step - loss: 0.1993 - acc: 0.0000e+00
Epoch 149/400
150/150 [==============================] - 0s 407us/step - loss: 0.1971 - acc: 0.0000e+00
Epoch 150/400
150/150 [==============================] - 0s 367us/step - loss: 0.1930 - acc: 0.0000e+00
Epoch 151/400
150/150 [==============================] - 0s 438us/step - loss: 0.1882 - acc: 0.0000e+00
Epoch 152/400
150/150 [==============================] - 0s 427us/step - loss: 0.1833 - acc: 0.0000e+00
Epoch 153/400
150/150 [==============================] - 0s 390us/step - loss:

In [13]:
inputY

[3.1789960861206055,
 3.7084450721740723,
 12.60128116607666,
 4.207683563232422,
 4.6168975830078125,
 3.0354909896850586,
 11.840925216674805,
 2.16172456741333,
 6.9489545822143555,
 7.532243251800537,
 7.60712194442749,
 2.8693666458129883,
 5.810518741607666,
 12.966415405273438,
 3.240056037902832,
 2.141334056854248,
 1.3585033416748047,
 2.926971435546875,
 2.0828723907470703,
 14.932662010192871,
 5.318228721618652,
 5.305410385131836,
 8.137553215026855,
 6.961904525756836,
 2.5756311416625977,
 4.247208595275879,
 1.539795994758606,
 11.89882755279541,
 7.541990756988525,
 11.527926445007324,
 8.602678298950195,
 6.296145439147949,
 4.302919387817383,
 2.9656925201416016,
 2.5696706771850586,
 6.573594570159912,
 2.178267478942871,
 1.4226915836334229,
 7.346165657043457,
 7.865504264831543,
 5.930924415588379,
 1.93234384059906,
 4.578503131866455,
 14.46782398223877,
 4.655867099761963,
 8.47146224975586,
 4.110506057739258,
 11.886197090148926,
 3.5231270790100098,
 7.994

In [14]:
inputX

[[[2.0, 0.05, 0.0],
  [2.0, 0.05, 15.0],
  [2.0, 0.05, 45.0],
  [3.46647047996521, 0.05, 95.0],
  [3.467799425125122, 0.05, 256.0],
  [3.425408124923706, 0.05, 276.0],
  [3.3799331188201904, 0.05, 326.0],
  [3.244380235671997, 0.05, 338.0]],
 [[3.0, 0.3, 0.0],
  [4.0, 0.3, 0.0],
  [4.0, 0.3, 0.0],
  [3.6977016925811768, 0.3, 0.0],
  [3.7350149154663086, 0.3, 0.0],
  [3.7670133113861084, 0.3, 0.0],
  [3.774385929107666, 0.3, 0.0],
  [3.7243993282318115, 0.3, 0.0]],
 [[13.0, 0.3, 0.0],
  [14.0, 0.3, 0.0],
  [15.0, 0.3, 0.0],
  [12.747862815856934, 0.3, 0.0],
  [12.772725105285645, 0.3, 0.0],
  [12.74567699432373, 0.3, 0.0],
  [12.563437461853027, 0.3, 0.0],
  [12.58956527709961, 0.3, 0.0]],
 [[6.0, 0.35, 0.0],
  [6.0, 0.35, 0.0],
  [6.0, 0.35, 94.0],
  [4.0640716552734375, 0.35, 185.0],
  [4.101159572601318, 0.35, 181.0],
  [4.120694160461426, 0.35, 211.0],
  [4.206335544586182, 0.35, 47.0],
  [4.145039081573486, 0.35, 211.0]],
 [[14.0, 0.35, 0.0],
  [9.0, 0.35, 0.0],
  [6.0, 0.35, 94.0]

In [22]:
matrix[0]

array([[ 1. ,  0.5,  0. ],
       [ 2. ,  0.5,  0. ],
       [ 5. ,  0.5, 25. ],
       [ 4. ,  0.5, 86. ],
       [ 7. ,  0.5, 32. ],
       [11. ,  0.5, 95. ],
       [ 8. ,  0.5, 74. ],
       [ 8. ,  0.5, 48. ],
       [ 7. ,  0.5, 79. ],
       [ 0. ,  0. ,  0. ]])